Idea for tuning:

need to specify which metrics need to be computed, which one needs to be minimized, and how large k is

* build k splits into train and test data
* for each train-test-split:
    * build a parameter grid using cls.tuning_params. (For each class we have to find out what a good grid would be.)
    * for each combination in the grid: 
        * translate api-response into f,m,u and store in the column gender_infered (GOT THIS FAR!!!)
        * compute all provided metrics on the resulting test_data DF restricted to training data
        * store relation grid-point:metrics
    * select grid_point which minimizes specified metric and compute all provided metrics on test set
    * store the metrics on test set
* compute average of metrics on all test sets for all metrics

In [1]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from evaluators import *
from evaluator import *
from itertools import product
from collections import OrderedDict
import numpy as np

In [2]:
data_source = 'all'
service_name = GenderizeIoEvaluator
evaluator = service_name(data_source)

In [3]:
evaluator.load_data(evaluated=True)

In [4]:
evaluator.test_data.head()

,first_name,middle_name,last_name,full_name,gender,origin,api_count,api_gender,api_name,api_probability,gender_infered
0,pierre,paul,grivel,pierre paul grivel,m,zbmath,5.0,male,pierre-paul,1.0,m
1,raul,,serapioni,raul serapioni,m,zbmath,821.0,male,raul,1.0,m
2,adriano,,moura,adriano moura,m,zbmath,166.0,male,adriano,0.99,m
3,ralf,,kieser,ralf kieser,m,zbmath,86.0,male,ralf,1.0,m
4,teppei,,ariyoshi,teppei ariyoshi,u,zbmath,,,teppei,,u


In [5]:
# we have to ignore 'u' in column 'gender'
evaluator.remove_rows_with_unknown_gender()

In [6]:
evaluator.tuning_params

('api_count', 'api_probability')

In [7]:
evaluator.build_parameter_grid([1,10, 50, 100], [0.5, 0.7, 0.8, 0.9])

[{'api_count': 1, 'api_probability': 0.5},
 {'api_count': 1, 'api_probability': 0.7},
 {'api_count': 1, 'api_probability': 0.8},
 {'api_count': 1, 'api_probability': 0.9},
 {'api_count': 10, 'api_probability': 0.5},
 {'api_count': 10, 'api_probability': 0.7},
 {'api_count': 10, 'api_probability': 0.8},
 {'api_count': 10, 'api_probability': 0.9},
 {'api_count': 50, 'api_probability': 0.5},
 {'api_count': 50, 'api_probability': 0.7},
 {'api_count': 50, 'api_probability': 0.8},
 {'api_count': 50, 'api_probability': 0.9},
 {'api_count': 100, 'api_probability': 0.5},
 {'api_count': 100, 'api_probability': 0.7},
 {'api_count': 100, 'api_probability': 0.8},
 {'api_count': 100, 'api_probability': 0.9}]

In [17]:
grid = evaluator.build_parameter_grid([1, 10, 20, 50, 100, 150], [0.5, 0.6, 0.7, 0.8, 0.9, 0.95])

In [9]:
# TODO: integrate into code for this specific evaluator
for col in evaluator.tuning_params:
    evaluator.test_data[col] = evaluator.test_data[col].replace({'':None}) # None is coded as empty string
    evaluator.test_data[col] = evaluator.test_data[col].astype(float)

In [18]:
# main method!!!
# compute predicted error
evaluator.compute_cv_score(5, grid, evaluator.compute_error_without_unknown)

[   0    1    2 ..., 5787 5788 5789] [   4    5   21 ..., 5774 5779 5785]
minimal train error: 0.0171067311268 corresponding test error: 0.024427480916
params for lowest train error: {'api_count': 100, 'api_probability': 0.7}
[   0    2    4 ..., 5786 5787 5789] [   1    3    6 ..., 5777 5784 5788]
minimal train error: 0.020081814801 corresponding test error: 0.012213740458
params for lowest train error: {'api_count': 100, 'api_probability': 0.7}
[   0    1    2 ..., 5786 5787 5788] [   7    9   10 ..., 5776 5780 5789]
minimal train error: 0.0194392523364 corresponding test error: 0.0149476831091
params for lowest train error: {'api_count': 100, 'api_probability': 0.7}
[   1    2    3 ..., 5785 5788 5789] [   0    8   16 ..., 5783 5786 5787]
minimal train error: 0.0187969924812 corresponding test error: 0.0175438596491
params for lowest train error: {'api_count': 100, 'api_probability': 0.7}
[   0    1    3 ..., 5787 5788 5789] [   2   14   15 ..., 5770 5773 5781]
minimal train error: 

0.018525524926310495